In [2]:
import rope.base.project
from rope.refactor.extract import ExtractMethod
import re

In [8]:

APOSTROPHE_MARKER = "__APOSTROPHE__"

def serialize(text: str):
    # Replace "'{var}'" with "__APOSTROPHE__{var}__APOSTROPHE__"
    return re.sub(r"'{(.*?)}'", f"{APOSTROPHE_MARKER}{{\\1}}{APOSTROPHE_MARKER}", text)


def deserialize(text: str):
    return re.sub(f"{APOSTROPHE_MARKER}{{(.*?)}}{APOSTROPHE_MARKER}", "'{\\1}'", text)


myproject = rope.base.project.Project('src')

myresource = myproject.get_resource('class_refactor.py')
contents = myresource.read()
serialized_contents = serialize(myresource.read())
myresource.write(serialized_contents)
extract_span = r'''2 * self.a'''
serialized_extract_span = serialize(extract_span)

start, end = serialized_contents.find(serialized_extract_span), serialized_contents.find(serialized_extract_span) + len(serialized_extract_span)
print(start, end)

try:
    extractor = ExtractMethod(myproject, myresource, start, end)
    change_set = extractor.get_changes("@helper")
    for change in change_set.changes:
        if change.old_contents is not None:
            change.old_contents = deserialize(change.old_contents)
        else:
            change.old_contents = deserialize(change.resource.read())
        change.new_contents = deserialize(change.new_contents)
    for change in change_set.changes:
        print(change.get_description())
except Exception as e:
    print(e)
finally:
    myresource.write(contents)

In [37]:
def extract_method(
    snippet,
    file_path,
    method_name,
    project_name="../../sweepai"
):
    project = rope.base.project.Project(project_name)

    resource = project.get_resource(file_path)
    contents = resource.read()
    start, end = contents.find(snippet), contents.find(snippet) + len(snippet)

    extractor = ExtractMethod(project, resource, start, end)
    change_set = extractor.get_changes(method_name, similar=True, global_=True)
    for change in change_set.changes:
        change.do()

    result = resource.read()

    for change in change_set.changes:
        change.undo()
    return result

snippet = r"""edit_sweep_comment(
                review_message + "\n\nSuccess! 🚀",
                4,
                pr_message=(
                    f"## Here's the PR! [{pr.html_url}]({pr.html_url}).\n{center(payment_message_start)}"
                ),
                done=True,
            )"""
print(extract_method(snippet, "handlers/on_ticket.py", "hydrate_sandbox"))